In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
import cv2
from tqdm import tqdm_notebook
from random import shuffle
import shutil
import pandas as pd

In [52]:
def organize_datasets(path_to_data, n=4000, ratio = 0.2):
    files = os.listdir(path_to_data)
    files = [os.path.join(path_to_data, f) for f in files]
    shuffle(files)
    files = files[:n]
    
    n = int(len(files) * ratio)
    
    val, train = files[:n], files[n:]
    
    #shutil.rmtree('./data/')
    #print('./data/ removed')
    
    
    for c in ['dogs', 'cats']:
        os.makedirs('./data/train/{0}'.format(c))
        os.makedirs('./data/validation/{0}'.format(c))
    
    print('folders created')
    
    for t in tqdm_notebook(train):
        if 'cats' in t:
            shuffle.copy2(t, os.path.join('.','data','train','cats'))
            
        else:
            shuffle.copy2(t, os.path.join('.','data','train','dogs'))
            
            
            
    for t in tqdm_notebook(val):
        if 'cats' in t:
            shuffle.copy2(t, os.path.join('.','data','val','cats'))
            
        else:
            shuffle.copy2(t, os.path.join('.','data','val','dogs'))
            
    
    print('Data copied')
    
    

In [53]:
organize_datasets(path_to_data = '../kaggle_dogcat/train/')

FileNotFoundError: [Errno 2] No such file or directory: '../kaggle_dogcat/train/'

In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras_tqdm import TQDMNotebookCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import MaxPooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import np_utils
from keras.callbacks import Callback


Using TensorFlow backend.
/Users/chibaryowei/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/chibaryowei/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/chibaryowei/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/chibaryowei/anaconda3/lib/pytho

In [3]:
batch_size = 32
n = 1400
ratio = 0.2
train_datagen = ImageDataGenerator(rescale=1/255.,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1/255.)

In [4]:
train_generator = train_datagen.flow_from_directory(
    './data/train/',
    target_size = (150, 150),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    './data/validation/',
    target_size = (150, 150),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 2540 images belonging to 2 classes.
Found 1460 images belonging to 2 classes.


In [9]:
train_generator

In [5]:
model = Sequential()

model.add(Conv2D(32,(3,3),input_shape = (150,150,3),padding = 'same',activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))


model.add(Conv2D(32,(3,3),padding = 'same',activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))


model.add(Conv2D(64,(3,3),padding = 'same',activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))


model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))


epochs = 30
learning_rate = 0.05
decay = learning_rate / epochs

sgd = SGD(lr = learning_rate, 
          momentum=0.8,
          decay=decay,
          nesterov=False)

model.compile(loss = 'binary_crossentropy', 
              optimizer = sgd,
              matrics = ['accuracy'])


model.summary()

W0814 14:57:42.027702 4496037312 deprecation_wrapper.py:119] From /Users/chibaryowei/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0814 14:57:42.051563 4496037312 deprecation_wrapper.py:119] From /Users/chibaryowei/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0814 14:57:42.056160 4496037312 deprecation_wrapper.py:119] From /Users/chibaryowei/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0814 14:57:42.085039 4496037312 deprecation_wrapper.py:119] From /Users/chibaryowei/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W08

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 18, 64)        0         
__________

In [6]:
# Callback for loss logging per epoch

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
    
    def on_epoch_end(self, batch, logs = {}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        

history = LossHistory()

## Callback for early stopping the training
earing_stopping = keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                              min_delta = 0,
                                              patience = 2,
                                              verbose = 0,
                                              mode = 'auto')


In [7]:
#mnist_model(0,[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])

In [8]:

#fitted_model = model.fit_generator(
#    train_generator,
#    steps_per_epoch=int(n*(1-ratio) // batch_size),
#    epochs=50,
#    validation_data =val_generator,
    #validation_steps=int(n*(1-ratio) // batch_size),
    #callbacks=[TQDMNotebookCallback()],
#    verbose = 0   
#)


fitted_model = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=50,
    verbose=0,
    validation_data=val_generator,
    validation_steps=100,
    callbacks = earing_stopping
)


ValueError: ('Some keys in session_kwargs are not supported at this time: %s', dict_keys(['matrics']))